In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 텐서 플로의 정보 출력 억제하기
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # GPU 장치 지정

# tf.debugging.set_log_device_placement(True)   # 이거 쓰지 마셈 ㅈㄴ 출력 더러움

if not tf.config.list_physical_devices('GPU'):
    print("감지된 GPU가 없습니다. GPU가 없으면 LSTM과 CNN이 매우 느릴 수 있습니다.")

## 11.
_연습문제: 최신 언어 모델 중 하나(예를 들어 BERT)로 세익스피어가 쓴 것 같은 텍스트를 생성해보세요._

최신 언어 모델을 사용하는 가장 간단한 방법은 허깅 페이스의 오픈 소스 라이브러리인 [트랜스포머스](https://huggingface.co/transformers/)를 사용하는 것입니다. 이 라이브러리는 자연어 처리(NLP)를 위한 최신 신경망 구조(BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet 등)와 사전훈련된 모델을 많이 제공합니다. 텐서플로와 파이토치를 지원합니다. 무엇보다도 사용하기 매우 쉽습니다.

먼저 사전훈련된 모델을 로드해 보죠. 이 예제에서 추가적인 언어 모델(입력 임베딩에 연결된 가중치를 가진 선형층)을 위에 얹은 OpenAI의 GPT 모델을 사용하겠습니다. 모델을 임포트하고 사전훈련된 가중치를 로드합니다(약 445MB의 데이터가 `~/.cache/torch/transformers`에 다운로드됩니다):

In [ ]:
from transformers import TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt")

그다음 이 모델을 위한 특별한 토크나이저(tokenizer)가 필요합니다. [spaCy](https://spacy.io/)와 [ftfy](https://pypi.org/project/ftfy/)가 설치되어 있으면 이를 사용하고 아니면 버트(Bert)의 `BasicTokenizer` 다음에 바이트-페어 인코딩(Byte-Pair Encoding)을 사용합니다(대부분의 경우에 잘 맞습니다).

In [ ]:
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")

이 토크나이저를 사용해 시작 텍스트를 토큰화하고 인코딩해 보죠:

In [ ]:
prompt_text = "This royal throne of kings, this sceptred isle"
encoded_prompt = tokenizer.encode(prompt_text,
                                  add_special_tokens=False,
                                  return_tensors="tf")
encoded_prompt

쉽군요! 그다음 이 모델을 사용해 시작 텍스트 다음에 이어지는 텍스트를 생성해 보겠습니다. 시작 텍스트 다음에 40개의 토큰을 이어서 다섯 개의 다른 문장을 생성합니다. 하이퍼파라미터에 대한 자세한 내용은 (허깅 페이스) Patrick von Platen의 [블로그](https://huggingface.co/blog/how-to-generate)를 참고하세요. 더 나은 결과를 얻기 위해 하이퍼파라미터를 조정해 볼 수 있습니다.

In [ ]:
num_sequences = 5
length = 40

generated_sequences = model.generate(
    input_ids=encoded_prompt,
    do_sample=True,
    max_length=length + len(encoded_prompt[0]),
    temperature=1.0,
    top_k=0,
    top_p=0.9,
    repetition_penalty=1.0,
    num_return_sequences=num_sequences,
)

generated_sequences

생성한 시퀀스를 디코딩하여 출력해 보죠:

In [ ]:
for sequence in generated_sequences:
    text = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(text)
    print("-" * 80)

GPT-2, CTRL, Transformer-XL, XLNet와 같이 더 최신의 (그리고 더 큰) 모델을 시도해 볼 수 있습니다. 다양한 언어 모델과 함께 트랜스포머스 라이브러리에 모두 사전훈련된 모델로 준비되어 있습니다. 모델마다 전처리 단계는 조금씩 다르므로 트랜스포머스 문서에 있는 [생성 예제](https://github.com/huggingface/transformers/blob/master/examples/run_generation.py)를 참고하세요(이 예제는 파이토치를 사용하지만 모델 클래스 이름의 시작 부분을 `TF`로 바꾸고 `.to()` 메서드 호출을 삭제하고 `"pt"` 대신에 `return_tensors="tf"`를 사용하면 텐서플로를 사용할 수 있습니다).